<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M1/notebooks/EDA_dataviz_application_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Knitr options
### Generic preamble
Sys.setenv(LANG = "en") # For english language
options(scipen = 5) # To deactivate annoying scientific number notation

# rm(list=ls()); graphics.off() # get rid of everything in the workspace
if (!require("knitr")) install.packages("knitr"); library(knitr) # For display of the markdown

### Knitr options
knitr::opts_chunk$set(warning=FALSE,
                     message=FALSE,
                     fig.align="center"
                     )

In [ ]:
### Load packages
library(tidyverse) # Collection of all the good stuff like dplyr, ggplot2 ect.
library(magrittr) # For extra-piping operators (eg. %<>%)



# Introduction

Welcome to the applied session in data visualization for Exploratory Data Analysis (EDA) in `R`.

# Introduction to `gglot2`

[`ggplot2`](https://ggplot2.tidyverse.org/) can be thought of as a mini-language (domain-specific language) within the `R` language. It is an R implementation of [Wilkinson's Grammar of Graphics book](https://www.springer.com/gp/book/9780387245447). [A Layered Grammar of Graphics](http://vita.had.co.nz/papers/layered-grammar.pdf) describes Hadley's implementation of these thoughts in the ggplot2's design. Due to its conceptional richness as well as the rich functionality provided, `ggplot2` has over time become the main sub-ecosustem for rgaphic visualization. Most packages dedicated to specialized forms of visualization (networks, interactions, etc.) will use the `ggplot` package as underlying plattform. So, it makes sense to dive a bit deeper into it functionality

Conceptually, the main idea behind the Grammar of Graphics is that a statistical graphic is a mapping from variables to aesthetic attributes (x axis value, y axis value, color, shape, size) of geometric objects (points, line, bars).

 While the Grammar of Graphic contains more elements, we will focus in this brief intro in the two main ones, aestetics and geometries.

* **Aestetics:** Devine the "surface" of your plot, in terms of what has to be mapped (size, coplor) on the x and y (and potentially adittional) axes. Aesteticts are defined within the `aes()` function.
* **Geometries:** Visual elements you can see in the plot itself, such as bars, lines, and points. They are defined within various `geom_XYZ()` functions.

![](https://github.com/SDS-AAU/SDS-master/raw/master/00_media/ggplot_structure.png)

Basically, you define a surface grid and then plot something on top. We will talk about all of that in depth in later sessions, for now that's all you need to know to understand the following simple examples.

# Application: the `BIXI Bikeshare Data` dataset

Lets take a step back and zoom a bit into different forms of visualization. We will now take a look at the `BIXI Bikeshare Data`, covering 500k bike-rides in the BIXI bike-sharing system in Montreal.



In [ ]:
bike <- readRDS(url("https://github.com/SDS-AAU/SDS-master/raw/master/00_data/bikes_montreal.rds?dl=1"))


Lets take a look:



In [ ]:
bike %>% glimpse()

In [ ]:
bike %>% head()


We see here a number of different variable types present, namely:

* Continuous variables
* Categorical variables
* Temporal variables

First of all: Lets remember, the first thing we do is defining the aestetics, first of all the dimensions (x, y) of the visualization.



In [ ]:
bike %>% ggplot(aes(x = weekday, y = start_hod))


The result will be an empty plane with the dimensions we defined. Note that there are more aestetic dimensions which can be used to convey informations visualy, such as for instance:

* Position (x, y)
* Color
* Shape
* Alpha (Transparency)

We will explore them later.

## Basic visualization of variable types

### Summaries of One Variable: Continuous

When attempting to summarize a single variable, histograms and density distributions are often the visualization of choice. We can do that easily by using the `geom_histogram()` layer. Notice that we only define a `x` aestetic, since we only summarize one variable



In [ ]:
bike %>% ggplot(aes(x = duration_sec)) +
  geom_histogram()


To plot a probability density function (PDF) instead, we can use the `geom_density()` layer.



In [ ]:
bike %>% ggplot(aes(x = duration_sec)) +
  geom_density()


Note the distribution appears right-skewed, since we have some outliers of very long bike rides. Adding a log-scale on the x-axis might help to reduce their impact on the visualization.




In [ ]:
bike %>% ggplot(aes(x = duration_sec)) +
  geom_histogram() +
  scale_x_log10()


In case we would already like to start looking at conditional distributions, we could add an adittional `fill` aestetic.



In [ ]:
bike %>% ggplot(aes(x = duration_sec, fill = weekday)) +
  geom_histogram() +
  scale_x_log10()


### Summaries of One Variable: Discrete

To do the same for a discrete variable, we would start with a simple barplot via `geom_bar()`. Notice again that we only define a x aestetic. `ggplot` per default will use the count on the y-axis.



In [ ]:
bike %>% ggplot(aes(x = start_dow)) +
  geom_bar()


We could also use the membership as fill aestetic to map further information in the plot.



In [ ]:
bike %>% ggplot(aes(x = start_dow, fill = membership)) +
  geom_bar()


### Summaries of One Variable: Temporal

A temporal variable can also be visualized as a line-plot with `geom_line()`.



In [ ]:
bike %>%
  count(start_wk) %>%
  ggplot(aes(x = start_wk, y = n)) +
  geom_line()


To instead (or in addition) add a trendline, we can use `geom_smooth()`



In [ ]:
bike %>%
  count(start_wk) %>%
  ggplot(aes(x = start_wk, y = n)) +
  geom_smooth()



## Summarizing multiple variablea jointly

Ok, that was pretty easy. However, the insights gained so far are pretty little. To tease out interesting pattern in our data, it might not be enough to only look at one variable at a time. To display relationships between multiple variables, we mainly can:

* Use aestetics such as `color`, `fill`, `size`, `shape` (alter the aestetics within one plot)
* Use `facet_wrap()`(produce  multiple plots)

Lets look at some examples:

First, we could take a look at the number of daily rides with  workweek / weekend days colored differently.



In [ ]:
# Compute daily counts & plot
bike %>%
  count(start_day, weekday) %>%
  ggplot(aes(start_day, n, color = weekday)) +
  geom_point()


Now let's look at how rides are distributed according to the time of day. Let's make a summary plot of weekly ride counts faceted by start hour of day and broken down by workweek/weekend. Here, we will use the `facet_grid`




In [ ]:
# Compute week_hod & plot
bike %>%
  count(start_wk, start_hod, weekday) %>%
  ggplot(aes(start_wk, n, color = weekday)) +
  geom_point() +
  facet_grid(~ start_hod) +
  scale_y_sqrt()


Expanding on the previous plot, let's add one more variable into our summary, adding a facet dimension for whether or not the rider is a member of BIXI.



In [ ]:
# Compute wk_memb_hod & plot
bike %>%
  count(start_wk, start_hod, weekday, membership) %>%
  ggplot(aes(start_wk, n, color = weekday)) +
  geom_point() +
  facet_grid(membership ~ start_hod) +
  scale_y_sqrt()


Let's now look at the number of rides vs. hour for each day. To start, we'll create a summary dataset for the first full month in the dataset (May) and look at it.



In [ ]:
# Compute daily_may &
bike %>%
  filter(start_mon == 5) %>%
  count(start_day, start_hod, membership) %>%
  ggplot(aes(start_hod, n, color = membership)) +
  geom_point() +
  facet_wrap(~ start_day, ncol = 7)


# Endnotes

### References

### Suggestions for further study

#### Own exploration
There is so much more to explore. However, since time is limited, I will leave it up to you to explore more.

* Take a moment to review the different geoms offered by `ggplot` [here](https://ggplot2.tidyverse.org/reference/index.html).
* For inspiration what can be done, check [here](http://r-statistics.co/Top50-Ggplot2-Visualizations-MasterList-R-Code.html).
* Check `ggplot2` addons  [here](https://www.ggplot2-exts.org/gallery/). Some of my favorite are:
   * `ggforce`: For a collection of adittional features
   * `patchwork`: For easy inegration of multiple plots jointly
   * `GGally`: Collection fo many cool plotting features, including many standard stats plot for correlation, distribution etc.
   * `ggmap`: For geoplotting
   * `ggraph`: For network plots (will be handled later)
   * `ggridges`: Ridge features, for example to create joy-plots
   * `ggalluvial`: For alluvial plots

#### Datacamp

* [Introduction to Data Visualization with ggplot2](https://learn.datacamp.com/courses/introduction-to-data-visualization-with-ggplot2): Really good and throrrough ggplot2 introduction. Rick has also more advanced ggplot courses for the ones who want to go deeper.
* [https://learn.datacamp.com/courses/communicating-with-data-in-the-tidyverse](Communicating with Data in the Tidyverse): Also includes more workflows for using dataviz for reporting & communicating to different audiences, eg. using Rmarkdown.
* [Interactive Data Visualization with plotly in R](https://learn.datacamp.com/courses/interactive-data-visualization-with-plotly-in-r): For the ones who want to go deeper into interactive plotting.

#### Other online courses

* [Data Visualization](https://datavizm20.classes.andrewheiss.com/): Amazing free online material, introducing you to many design concepts and reflections on dataviz, combined with hands-on examples.
* [DataCarpentry: Data Visualization with ggplot2](https://datacarpentry.org/R-ecology-lesson/04-visualization-ggplot2.html#Plotting_with_ggplot2): Generally good course. You might here enjoy the attention paid to statistical plots.

#### Papers, Ebooks & chapters
* R for Data Science (Grolemund & Wickham)
   * [Chapter 3](https://r4ds.had.co.nz/data-visualisation.html): Basics of DataViz
   * [Chapter 7](https://r4ds.had.co.nz/exploratory-data-analysis.html): DataViz for EDA
   * [Chapter 28](https://r4ds.had.co.nz/graphics-for-communication.html): Finetuning DataViz
* [Wickham, Hadley. "A layered grammar of graphics." Journal of Computational and Graphical Statistics 19.1 (2010): 3-28.](http://vita.had.co.nz/papers/layered-grammar.pdf): General concept of the grammar of graphics
* Healy, Kieran. Data visualization: a practical introduction. Princeton University Press, 2018. [online available here](https://socviz.co/index.html#preface): Good inrto with many best-practice advices.


### Session Info


In [ ]:
sessionInfo()